In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
from scipy.spatial import cKDTree
from math import *
np.random.seed(123)
import urllib.request
import urllib, os

In [2]:
import pandas as pd
import xarray as xr
import math
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [3]:
import numpy as np
import xarray as xr
import xesmf as xe

In [4]:
df_prep = xr.open_dataset(r'/home/shubham/ugp/cleaned/Land_and_sea_temp_1850-2022.nc')

In [5]:
df_prep

<xarray.Dataset>
Dimensions:      (longitude: 360, latitude: 180, time: 2064, month_number: 12)
Coordinates:
  * longitude    (longitude) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * latitude     (latitude) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * time         (time) datetime64[ns] 1850-02-01 1850-03-01 ... 2022-01-01
Dimensions without coordinates: month_number
Data variables:
    land_mask    (latitude, longitude) float64 ...
    temperature  (time, latitude, longitude) float32 ...
    climatology  (month_number, latitude, longitude) float32 ...
Attributes:
    Conventions:  Berkeley Earth Internal Convention (based on CF-1.5)
    title:        Native Format Berkeley Earth Surface Temperature Anomaly Field
    institution:  Berkeley Earth Surface Temperature Project
    comment:      This file contains surface temperature anomaly field gridde...

In [6]:
df_prep["slice"] = df_prep["temperature"].isel(time=0)
df_prep

<xarray.Dataset>
Dimensions:      (longitude: 360, latitude: 180, time: 2064, month_number: 12)
Coordinates:
  * longitude    (longitude) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * latitude     (latitude) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * time         (time) datetime64[ns] 1850-02-01 1850-03-01 ... 2022-01-01
Dimensions without coordinates: month_number
Data variables:
    land_mask    (latitude, longitude) float64 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0
    temperature  (time, latitude, longitude) float32 ...
    climatology  (month_number, latitude, longitude) float32 ...
    slice        (latitude, longitude) float32 ...
Attributes:
    Conventions:  Berkeley Earth Internal Convention (based on CF-1.5)
    title:        Native Format Berkeley Earth Surface Temperature Anomaly Field
    institution:  Berkeley Earth Surface Temperature Project
    comment:      This file contains surface temperature anomaly field gridde...

In [7]:
df_prep = df_prep.rename({'longitude':'lon', 'latitude':'lat'})

In [8]:
df_prep

<xarray.Dataset>
Dimensions:      (lon: 360, lat: 180, time: 2064, month_number: 12)
Coordinates:
  * lon          (lon) float32 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat          (lat) float32 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time         (time) datetime64[ns] 1850-02-01 1850-03-01 ... 2022-01-01
Dimensions without coordinates: month_number
Data variables:
    land_mask    (lat, lon) float64 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0 0.0
    temperature  (time, lat, lon) float32 ...
    climatology  (month_number, lat, lon) float32 ...
    slice        (lat, lon) float32 nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    Conventions:  Berkeley Earth Internal Convention (based on CF-1.5)
    title:        Native Format Berkeley Earth Surface Temperature Anomaly Field
    institution:  Berkeley Earth Surface Temperature Project
    comment:      This file contains surface temperature anomaly field gridde...

In [9]:
df_prep_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-90,90, 2.0)),
        "lon": (["lon"], np.arange(0, 360, 2)),
    }
)

regridder = xe.Regridder(df_prep, df_prep_out, 'bilinear')
regridder

Overwrite existing file: bilinear_180x360_90x180.nc 
 You can set reuse_weights=True to save computing time.


xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_180x360_90x180.nc 
Reuse pre-computed weights? False 
Input grid shape:           (180, 360) 
Output grid shape:          (90, 180) 
Output grid dimension name: ('lat', 'lon') 
Periodic in longitude?      False

In [10]:
df_out = regridder(df_prep)
df_out

using dimensions ('lat', 'lon') from data variable land_mask as the horizontal dimensions for this dataset.


/home/shubham/anaconda3/envs/ugp_env/lib/python3.7/site-packages/xesmf/frontend.py:486: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs


<xarray.Dataset>
Dimensions:      (lat: 90, lon: 180, time: 2064, month_number: 12)
Coordinates:
  * time         (time) datetime64[ns] 1850-02-01 1850-03-01 ... 2022-01-01
  * lon          (lon) int64 0 2 4 6 8 10 12 14 ... 346 348 350 352 354 356 358
  * lat          (lat) float64 -90.0 -88.0 -86.0 -84.0 ... 82.0 84.0 86.0 88.0
Dimensions without coordinates: month_number
Data variables:
    land_mask    (lat, lon) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    temperature  (time, lat, lon) float64 0.0 0.0 0.0 0.0 ... 3.6 3.568 3.526
    climatology  (month_number, lat, lon) float64 0.0 0.0 0.0 ... -30.52 -30.46
    slice        (lat, lon) float64 0.0 0.0 0.0 0.0 ... 0.5929 0.6085 0.6243
Attributes:
    regrid_method:  bilinear

In [11]:
df_out = df_out.drop(['slice','climatology','land_mask'])

In [12]:
df_out['time'][1826:-2]

<xarray.DataArray 'time' (time: 236)>
array(['2002-04-01T00:00:00.000000000', '2002-05-01T00:00:00.000000000',
       '2002-06-01T00:00:00.000000000', ..., '2021-09-01T00:00:00.000000000',
       '2021-10-01T00:00:00.000000000', '2021-11-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01

In [13]:
df_out = df_out['temperature'][:][:][1826:-2]

In [14]:
pd.to_datetime(df_out['time'])[0]

Timestamp('2002-04-01 00:00:00')

In [15]:
new_index = pd.period_range(pd.to_datetime(df_out['time'])[0],pd.to_datetime(df_out['time'])[-1],freq='M')

In [16]:
new_index = pd.DataFrame(new_index)
new_index

,0
0,2002-04
1,2002-05
2,2002-06
3,2002-07
4,2002-08
...,...
231,2021-07
232,2021-08
233,2021-09
234,2021-10


In [17]:
df_out

<xarray.DataArray 'temperature' (time: 236, lat: 90, lon: 180)>
array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 3.95240056,  3.94822195,  3.9404967 , ...,  3.9361075 ,
          3.94313377,  3.94931398],
        [ 3.3263069 ,  3.33146533,  3.33729676, ...,  3.31661186,
          3.31980875,  3.32257463],
        ...,
        [ 1.33779894,  1.26459526,  1.19912074, ...,  1.49517117,
          1.44821064,  1.39894087],
        [ 2.0484515 ,  2.05731672,  2.07266953, ...,  2.02272286,
          2.03452857,  2.03755767],
        [ 2.43877849,  2.45348422,  2.47399822, ...,  2.41190429,
          2.41841313,  2.42746171]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 3.89695339,  3.90375439,  3.90885078, ...,  3.85966253,
          3.87339563,  3.88624297],
        [ 3.74702462,  3.75982086,  3.77276791, ...,  3.70871544,
          3.72206212,  3.73457666],
...
        [ 3.50597774,  3.49788954,  3.48865581, ...,  3.52925481,
          3.53272728,  3.51773954],
        [ 3.27428018,  3.27225083,  3.26938866, ...,  3.30782188,
          3.29907807,  3.29129017],
        [ 3.14745591,  3.14083824,  3.14095868, ...,  3.14970284,
          3.15019069,  3.14995841]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-1.69178333, -1.6991943 , -1.70428833, ..., -1.65202978,
         -1.66652162, -1.68016118],
        [-1.67822124, -1.69826499, -1.71794325, ..., -1.61638949,
         -1.63785427, -1.65817323],
        ...,
        [ 3.00082476,  2.96489298,  2.91359545, ...,  3.13799879,
          3.10200051,  3.06342812],
        [ 3.21549209,  3.19120568,  3.16592174, ...,  3.34383772,
          3.30436696,  3.25931042],
        [ 3.36895318,  3.35233328,  3.3622302 , ...,  3.43053567,
          3.41202395,  3.38954589]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [18]:
type(new_index[0][0])

pandas._libs.tslibs.period.Period

In [19]:
new_index = new_index[0].dt.to_timestamp()
type(new_index)

pandas.core.series.Series

In [20]:
new_index

0     2002-04-01
1     2002-05-01
2     2002-06-01
3     2002-07-01
4     2002-08-01
         ...    
231   2021-07-01
232   2021-08-01
233   2021-09-01
234   2021-10-01
235   2021-11-01
Name: 0, Length: 236, dtype: datetime64[ns]

In [21]:
df_out['time'] = pd.to_datetime(df_out['time']).strftime("%Y-%m")

In [22]:
df_out['time'] = pd.to_datetime(df_out['time'])
df_out

<xarray.DataArray 'temperature' (time: 236, lat: 90, lon: 180)>
array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 3.95240056,  3.94822195,  3.9404967 , ...,  3.9361075 ,
          3.94313377,  3.94931398],
        [ 3.3263069 ,  3.33146533,  3.33729676, ...,  3.31661186,
          3.31980875,  3.32257463],
        ...,
        [ 1.33779894,  1.26459526,  1.19912074, ...,  1.49517117,
          1.44821064,  1.39894087],
        [ 2.0484515 ,  2.05731672,  2.07266953, ...,  2.02272286,
          2.03452857,  2.03755767],
        [ 2.43877849,  2.45348422,  2.47399822, ...,  2.41190429,
          2.41841313,  2.42746171]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 3.89695339,  3.90375439,  3.90885078, ...,  3.85966253,
          3.87339563,  3.88624297],
        [ 3.74702462,  3.75982086,  3.77276791, ...,  3.70871544,
          3.72206212,  3.73457666],
...
        [ 3.50597774,  3.49788954,  3.48865581, ...,  3.52925481,
          3.53272728,  3.51773954],
        [ 3.27428018,  3.27225083,  3.26938866, ...,  3.30782188,
          3.29907807,  3.29129017],
        [ 3.14745591,  3.14083824,  3.14095868, ...,  3.14970284,
          3.15019069,  3.14995841]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-1.69178333, -1.6991943 , -1.70428833, ..., -1.65202978,
         -1.66652162, -1.68016118],
        [-1.67822124, -1.69826499, -1.71794325, ..., -1.61638949,
         -1.63785427, -1.65817323],
        ...,
        [ 3.00082476,  2.96489298,  2.91359545, ...,  3.13799879,
          3.10200051,  3.06342812],
        [ 3.21549209,  3.19120568,  3.16592174, ...,  3.34383772,
          3.30436696,  3.25931042],
        [ 3.36895318,  3.35233328,  3.3622302 , ...,  3.43053567,
          3.41202395,  3.38954589]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [23]:
df_out['time'][:]

<xarray.DataArray 'time' (time: 236)>
array(['2002-04-01T00:00:00.000000000', '2002-05-01T00:00:00.000000000',
       '2002-06-01T00:00:00.000000000', ..., '2021-09-01T00:00:00.000000000',
       '2021-10-01T00:00:00.000000000', '2021-11-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01

In [24]:
new_index

0     2002-04-01
1     2002-05-01
2     2002-06-01
3     2002-07-01
4     2002-08-01
         ...    
231   2021-07-01
232   2021-08-01
233   2021-09-01
234   2021-10-01
235   2021-11-01
Name: 0, Length: 236, dtype: datetime64[ns]

In [25]:
df_out['lat']

<xarray.DataArray 'lat' (lat: 90)>
array([-90., -88., -86., -84., -82., -80., -78., -76., -74., -72., -70., -68.,
       -66., -64., -62., -60., -58., -56., -54., -52., -50., -48., -46., -44.,
       -42., -40., -38., -36., -34., -32., -30., -28., -26., -24., -22., -20.,
       -18., -16., -14., -12., -10.,  -8.,  -6.,  -4.,  -2.,   0.,   2.,   4.,
         6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,  22.,  24.,  26.,  28.,
        30.,  32.,  34.,  36.,  38.,  40.,  42.,  44.,  46.,  48.,  50.,  52.,
        54.,  56.,  58.,  60.,  62.,  64.,  66.,  68.,  70.,  72.,  74.,  76.,
        78.,  80.,  82.,  84.,  86.,  88.])
Coordinates:
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [26]:
df_out

<xarray.DataArray 'temperature' (time: 236, lat: 90, lon: 180)>
array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 3.95240056,  3.94822195,  3.9404967 , ...,  3.9361075 ,
          3.94313377,  3.94931398],
        [ 3.3263069 ,  3.33146533,  3.33729676, ...,  3.31661186,
          3.31980875,  3.32257463],
        ...,
        [ 1.33779894,  1.26459526,  1.19912074, ...,  1.49517117,
          1.44821064,  1.39894087],
        [ 2.0484515 ,  2.05731672,  2.07266953, ...,  2.02272286,
          2.03452857,  2.03755767],
        [ 2.43877849,  2.45348422,  2.47399822, ...,  2.41190429,
          2.41841313,  2.42746171]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 3.89695339,  3.90375439,  3.90885078, ...,  3.85966253,
          3.87339563,  3.88624297],
        [ 3.74702462,  3.75982086,  3.77276791, ...,  3.70871544,
          3.72206212,  3.73457666],
...
        [ 3.50597774,  3.49788954,  3.48865581, ...,  3.52925481,
          3.53272728,  3.51773954],
        [ 3.27428018,  3.27225083,  3.26938866, ...,  3.30782188,
          3.29907807,  3.29129017],
        [ 3.14745591,  3.14083824,  3.14095868, ...,  3.14970284,
          3.15019069,  3.14995841]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-1.69178333, -1.6991943 , -1.70428833, ..., -1.65202978,
         -1.66652162, -1.68016118],
        [-1.67822124, -1.69826499, -1.71794325, ..., -1.61638949,
         -1.63785427, -1.65817323],
        ...,
        [ 3.00082476,  2.96489298,  2.91359545, ...,  3.13799879,
          3.10200051,  3.06342812],
        [ 3.21549209,  3.19120568,  3.16592174, ...,  3.34383772,
          3.30436696,  3.25931042],
        [ 3.36895318,  3.35233328,  3.3622302 , ...,  3.43053567,
          3.41202395,  3.38954589]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [27]:
x = df_out.reindex({"time": new_index})
x

<xarray.DataArray 'temperature' (time: 236, lat: 90, lon: 180)>
array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 3.95240056,  3.94822195,  3.9404967 , ...,  3.9361075 ,
          3.94313377,  3.94931398],
        [ 3.3263069 ,  3.33146533,  3.33729676, ...,  3.31661186,
          3.31980875,  3.32257463],
        ...,
        [ 1.33779894,  1.26459526,  1.19912074, ...,  1.49517117,
          1.44821064,  1.39894087],
        [ 2.0484515 ,  2.05731672,  2.07266953, ...,  2.02272286,
          2.03452857,  2.03755767],
        [ 2.43877849,  2.45348422,  2.47399822, ...,  2.41190429,
          2.41841313,  2.42746171]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 3.89695339,  3.90375439,  3.90885078, ...,  3.85966253,
          3.87339563,  3.88624297],
        [ 3.74702462,  3.75982086,  3.77276791, ...,  3.70871544,
          3.72206212,  3.73457666],
...
        [ 3.50597774,  3.49788954,  3.48865581, ...,  3.52925481,
          3.53272728,  3.51773954],
        [ 3.27428018,  3.27225083,  3.26938866, ...,  3.30782188,
          3.29907807,  3.29129017],
        [ 3.14745591,  3.14083824,  3.14095868, ...,  3.14970284,
          3.15019069,  3.14995841]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-1.69178333, -1.6991943 , -1.70428833, ..., -1.65202978,
         -1.66652162, -1.68016118],
        [-1.67822124, -1.69826499, -1.71794325, ..., -1.61638949,
         -1.63785427, -1.65817323],
        ...,
        [ 3.00082476,  2.96489298,  2.91359545, ...,  3.13799879,
          3.10200051,  3.06342812],
        [ 3.21549209,  3.19120568,  3.16592174, ...,  3.34383772,
          3.30436696,  3.25931042],
        [ 3.36895318,  3.35233328,  3.3622302 , ...,  3.43053567,
          3.41202395,  3.38954589]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2021-11-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [28]:
x.to_netcdf('/home/shubham/ugp/cl_new/regrid_2_deg/Land_and_sea_temp_04-2002_11-2021.nc')